In [1]:
!pip install langchain openai langchain-openai langchain-community -q


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [35]:
import re
import pandas as pd
from tqdm import tqdm
from getpass import getpass

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, TransformChain
from langchain.output_parsers import ResponseSchema, StructuredOutputParser 
from langchain.schema.output_parser import StrOutputParser

## Если используете ключ от OpenAI, запустите эту ячейку 👇

In [ ]:
import os
from langchain_openai import ChatOpenAI


# os.environ['OPENAI_API_KEY'] = "Введите ваш OpenAI API ключ"
os.environ['OPENAI_API_KEY'] = getpass(prompt='Введите ваш OpenAI API ключ')

# Инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0)

## Если используете ключ из курса, запустите эти ячейки 👇


In [ ]:
!wget https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py

In [3]:
from utils import ChatOpenAI
from getpass import getpass

#course_api_key= "Введите ваш API ключ, полученный в боте курса"
course_api_key = getpass(prompt='Введите ваш API ключ, полученный в боте курса')

# инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)

## Задание 3.2.9 🤔 Кажется, это что-то на LLM-ском? 🧐

In [4]:
df = pd.read_csv("https://stepik.org/media/attachments/lesson/1110883/raw_texts.csv")
df.head()

,raw_text
0,"The sun was setting, casting long shadows over..."
1,"Le soleil se couchait, jetant de longues ombre..."
2,"El sol se estaba poniendo, proyectando largas ..."
3,"La ciudad estaba llena de vida, sus calles lle..."
4,"La ville était pleine de vie, ses rues remplie..."


Напишем функцию, которая очистит текст от ненужных символов: `¿, ¡, £`

In [5]:
def clean_text(inputs: dict) -> dict:
    text = inputs["text"]

    text = re.sub(r'[¿¡£]', '', text)

    return {"text": text}

Будем просить у модели определять язык и имя главного персонажа и выдавать ответ в виде словаря. Для этого создадим `Output parser`, с которым вы уже познакомились в прошлых уроках.

In [9]:
# Определим схемы ответа
language_schema = ResponseSchema(
    name="language",
    description="What language is the text in? Answer with the name of the language.",
    )

person_schema = ResponseSchema(
    name="main_character",
    description="""Find the main character of the text. 
    Answer with the name of the character in language of the text.
    """
    )

response_schemas = [language_schema, person_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas) # Создаём парсер и подаём в него список со схемами
format_instructions = output_parser.get_format_instructions() # Получаем инструкции по форматированию ответа

Напишем шаблон промпта со своим вопросом и инструкциями по форматированию ответа. Будем передавать в этот промпт сырой текст

In [18]:
# создайте цепочку диалога, в параметры передайте языковую модель и память
template = """You are a helpful assistant. 
You will do your best to extract the answers to the question from the text.
You have to carefully read the text and find what language 
its written in and the main character of the text.

text: {text}

{format_instructions}"""

prompt = PromptTemplate(
    input_variables=["text"], template=template,
    partial_variables={"format_instructions": format_instructions},
)


Создадим цепочку с помощью `LCEL`

In [45]:
chain = clean_text | prompt | llm | output_parser

In [61]:
clean_text({"text": text})

dict

In [64]:
result = []
for text in tqdm(df['raw_text']):
    # YOUR CODE HERE
    result.append(clean_text({"text": text}))
    result[-1].update(chain.invoke({"text": text}))
    # break # Для отладки. Уберите, когда убедитесь, что на одном примере работает

100%|██████████| 13/13 [00:17<00:00,  1.34s/it]


In [67]:
pd.DataFrame(result)[['text', 'language', 'main_character']]

,text,language,main_character
0,"The sun was setting, casting long shadows over...",English,John
1,"Le soleil se couchait, jetant de longues ombre...",French,Pierre
2,"El sol se estaba poniendo, proyectando largas ...",Spanish,Carlos
3,"La ciudad estaba llena de vida, sus calles lle...",Spanish,Juan
4,"La ville était pleine de vie, ses rues remplie...",French,Jean
5,"Die Stadt war voller Leben, ihre Straßen gefül...",German,Johann
6,Die Sonne ging unter und warf lange Schatten ü...,German,Hans
7,"В тихом уголке старого города, где узкие улочк...",Russian,Анна
8,In a small town nestled between the mountains ...,English,Laura
9,En un pequeño pueblo situado entre las montaña...,Spanish,Maria


Сохраним всё в итоговый файл. Убедитесь, что на этом этапе у вас в столбцах

- `text` - очищенный текст (без символов ¿, ¡, £)
- `language` - язык, на котором написан текст (название языка указать на английском языке)
- `main_character` - имя главного персонажа в тексте (указать на том языке, на котором и написан сам текст)

In [68]:
pd.DataFrame(result)[['text', 'language', 'main_character']].to_csv('3.2.9_solution.csv', index=False)